# Business Information

In [1]:
import pandas as pd
import numpy as np
import datetime  
import matplotlib.pyplot as plt
import re

In [2]:
# read the data to a dataframe
df1 = pd.read_csv('data/mergedData022522clean.csv')
df2 = pd.read_csv('data/mergedData033122clean.csv')
df3 = pd.read_csv('data/mergedData041022clean.csv')

In [3]:
# FYI DELETE LATER
df1.columns

Index(['Unnamed: 0', 'listing_id', 'shop_id', 'title_listing', 'description',
       'num_favorers_listing', 'listing_type', 'tags', 'materials', 'style',
       'taxonomy_id', 'price.amount', 'price.divisor', 'price.currency_code',
       'creation_timestamp', 'ending_timestamp', 'original_creation_timestamp',
       'quantity', 'shop_name', 'create_date', 'title_shop', 'announcement',
       'currency_code', 'listing_active_count', 'accepts_custom_requests',
       'url', 'num_favorers_shop', 'languages', 'is_shop_us_based',
       'transaction_sold_count', 'shipping_from_country_iso',
       'shop_location_country_iso', 'review_average', 'review_count'],
      dtype='object')

In [4]:
# create a price column by dividing price amount by price divisor (not sure why ETSY splits this up)
df1['price'] = df1['price.amount']/df1['price.divisor']
df2['price'] = df2['price.amount']/df2['price.divisor']
df3['price'] = df3['price.amount']/df3['price.divisor']

## Dates the data covers

#### TO DO: Make this a function

In [5]:
# date range of df1
first = df1["creation_timestamp"].min()
last = df1["creation_timestamp"].max()
print(datetime.date.fromtimestamp(first), datetime.date.fromtimestamp(last))

2022-02-07 2022-02-25


In [6]:
# date range of df2
first = df2["creation_timestamp"].min()
last = df2["creation_timestamp"].max()
print(datetime.date.fromtimestamp(first), datetime.date.fromtimestamp(last))

2022-03-14 2022-03-31


In [7]:
# date range of df3
first = df3["creation_timestamp"].min()
last = df3["creation_timestamp"].max()
print(datetime.date.fromtimestamp(first), datetime.date.fromtimestamp(last))

2022-03-23 2022-04-10


### Identify top products listed

In [8]:
# I used the top tags and title word counts in the text analysis to manually identify common product types:
products = [
'mug', 
'cup', 
'plate', 
'vase', 
'bowl', 
'tea cup', 
'dinnerware set', 
'soap dish', 
'utensil holder', 
'candle holder',
'ring holder',
'serving bowl', 
'jar', 
'spoon rest', 
'planter',
'pendant',
'jug',
'soap dispenser',
'tumbler'   
]

### Average Prices for Top Products in US Dollars

In [9]:
# dataframe with listings that have prices in US Dollars
usd1 = df1[df1['price.currency_code'] == "USD"].reset_index(drop=True)
usd2 = df2[df2['price.currency_code'] == "USD"].reset_index(drop=True)
usd3 = df3[df3['price.currency_code'] == "USD"].reset_index(drop=True)

In [10]:
# this function adds a column for each item in the products list and assigns the value 1 if the product appears in the tags
def add_product_columns(dataframe, list):
    for p in list:
        dataframe[p] = pd.Series(dtype='int')
        for i in range (0, len(dataframe)):
            if ((p in dataframe['tags'][i]) == True):
                dataframe[p][i] = 1

In [11]:
# use the function to add the columns
add_product_columns(usd1, products)
add_product_columns(usd2, products)
add_product_columns(usd3, products)

<ipython-input-10-ea799e53fde6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[p][i] = 1


In [12]:
# this function calculates the average price for each product in the list
def calculate_product_avg(dataframe, prods):  
    average_prices = [] 
    for i in range(0, len(prods)):
        p = prods[i]
        df = dataframe[dataframe[p] == 1]
        avg = df['price'].mean()
        average_prices.append(avg)
    return average_prices

In [13]:
avg_prices_usd1 = calculate_product_avg(usd1, products)
avg_prices_usd2 = calculate_product_avg(usd2, products)
avg_prices_usd3 = calculate_product_avg(usd3, products)

In [15]:
# create empty dataframe for average prices
average_prices_df = pd.DataFrame(columns = products)

Fix this:

In [17]:
average_prices_df.loc[len(average_prices_df)+1] = avg_prices_usd1

In [18]:
average_prices_df.loc[len(average_prices_df)+1] = avg_prices_usd2

In [19]:
average_prices_df.loc[len(average_prices_df)+1] = avg_prices_usd3

In [20]:
average_prices_df

,mug,cup,plate,vase,bowl,tea cup,dinnerware set,soap dish,utensil holder,candle holder,ring holder,serving bowl,jar,spoon rest,planter,pendant,jug,soap dispenser,tumbler
1,41.186386,39.851983,93.173459,67.651238,67.126415,39.562961,243.736923,24.086897,43.953125,34.533226,29.575385,72.850696,65.549196,20.645669,42.507099,30.897656,47.727627,38.032500,33.993988
2,41.611503,40.502263,69.610158,72.236383,61.521238,42.247560,137.076809,27.956386,56.066923,40.610909,27.384583,73.489950,67.175242,22.241500,40.498852,18.051875,45.615970,31.740417,38.876074
3,42.024441,42.047080,72.586576,67.167079,65.485365,44.225438,144.104468,27.671368,56.263333,37.096923,28.314444,82.741036,76.859959,22.844899,43.473366,18.675397,126.598113,31.572553,39.234161


In [22]:
# Average Price for all the listings in USD
usd1['price'].mean()

56.87491080348218

### Business Questions

Characteristics of a successful shop?  (High Ratings and high number of reviews)

Emerging trends/New Market Opportunites - compare to more recent set of listings

Identify highest selling product categories

Identify: Top companies, top products

Model - Predict products based on text?  Hierarchical clustering?  TSNE w/dbscan

If you can get product clusters - overlay pricing?